In [3]:
import clickhouse_connect
import polars as pl

Use the following command to get the IP Address of the docker container running the database, and then
use that for the host argument below:
docker inspect -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' database

In [4]:
# client = clickhouse_connect.get_client(host='172.20.0.2', username='default', password='Password123!')
client = clickhouse_connect.get_client(host='hub.publichealthhq.xyz', port=18123, username='default', password='Password123!')

DatabaseError: :HTTPDriver for http://hub.publichealthhq.xyz:18123 returned response code 403)
 Code: 516. DB::Exception: default: Authentication failed: password is incorrect, or there is no user with such name.

If you have installed ClickHouse and forgot password you can reset it in the configuration file.
The password for default user is typically located at /etc/clickhouse-server/users.d/default-password.xml
and deleting this file will reset the password.
See also /etc/clickhouse-server/users.xml on the server where ClickHouse is installed.

. (AUTHENTICATION_FAILED) (version 24.1.5.6 (official build))


In [3]:
result = client.query("SELECT YEAR, MONTH, MONTH_NAME, STATEFIP, STATEFIP_NAME, COUNTY, METAREA_NAME, EDUC_NAME, HIGRADE_NAME, FTOTVAL, INCTOT FROM cps_00004.cps_00004 LIMIT 1000000")

In [4]:
next(result.named_results())

{'YEAR': 2005,
 'MONTH': 3,
 'MONTH_NAME': 'March',
 'STATEFIP': 48,
 'STATEFIP_NAME': '48',
 'COUNTY': 0,
 'METAREA_NAME': 'Houston-Baytown-Sugar Land, TX',
 'EDUC_NAME': 'Grade 9',
 'HIGRADE_NAME': '',
 'FTOTVAL': 0,
 'INCTOT': 0}

In [5]:
df = pl.from_dicts(result.named_results(), infer_schema_length=400)

In [6]:
df

YEAR,MONTH,MONTH_NAME,STATEFIP,STATEFIP_NAME,COUNTY,METAREA_NAME,EDUC_NAME,HIGRADE_NAME,FTOTVAL,INCTOT
i64,i64,str,i64,str,i64,str,str,str,i64,i64
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grade 9""","""""",0,0
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grade 9""","""""",1800,1800
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Associate's de…","""""",42000,22000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""High school di…","""""",42000,20000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Some college b…","""""",7056,7056
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""NIU or blank""","""""",7056,999999999
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""High school di…","""""",32000,32000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Master's degre…","""""",52288,52288
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grades 5 or 6""","""""",8400,8400


In [7]:
# Filter out the codes related to missing data
df = df.filter(pl.col('INCTOT') != 999999999).filter(pl.col('INCTOT')  != 999999998)

In [8]:
df

YEAR,MONTH,MONTH_NAME,STATEFIP,STATEFIP_NAME,COUNTY,METAREA_NAME,EDUC_NAME,HIGRADE_NAME,FTOTVAL,INCTOT
i64,i64,str,i64,str,i64,str,str,str,i64,i64
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grade 9""","""""",0,0
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grade 9""","""""",1800,1800
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Associate's de…","""""",42000,22000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""High school di…","""""",42000,20000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Some college b…","""""",7056,7056
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""High school di…","""""",32000,32000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Master's degre…","""""",52288,52288
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grades 5 or 6""","""""",8400,8400
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Bachelor's deg…","""""",8400,0


In [9]:
grouped = df.group_by(["EDUC_NAME"]).agg(pl.col('INCTOT').median()).sort('INCTOT')



In [10]:
for (educ, inc) in grouped.rows():
    print(f"{educ}: {inc}")

Grade 9: 828.5
Grade 7: 1116.0
: 1200.0
Grade 8: 1512.0
Grade 1: 1571.0
None or preschool: 1700.0
Grade 10: 1820.0
Grade 3: 1848.0
Grade 2: 1860.0
Grade 5: 1900.0
Grade 4: 1949.0
Grade 6: 2100.0
Grade 11: 2220.0
12th grade, diploma unclear: 5035.0
1 year of college: 5860.0
3 years of college: 6000.0
Grades 7 or 8: 6371.0
2 years of college: 7500.0
Grades 1, 2, 3, or 4: 8587.5
12th grade, no diploma: 8722.5
Grades 5 or 6: 10000.0
4 years of college: 10500.0
5 years of college: 11208.5
High school diploma or equivalent: 14858.0
Some college but no degree: 19215.0
6+ years of college: 19671.0
Associate's degree, occupational/vocational program: 26013.5
Associate's degree, academic program: 28000.0
Bachelor's degree: 35000.0
Master's degree: 46000.0
Doctorate degree: 62826.0
Professional school degree: 70039.0
